# 경사 하강법을 이용한 얕은 신경망 학습


In [11]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [12]:
EPOCHS = 1000

## 네트워크 구조 정의
### 얕은 신경망
#### 입력 계층 : 2, 은닉 계층 : 128 (Sigmoid activation), 출력 계층 : 10 (Softmax activation)

In [13]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128, input_dim=2, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
            
    def call(self, x, training=None, mask=None):
        x = self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [14]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables) # df(x)/dx
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_metric(labels, predictions)

## 데이터셋 생성, 전처리

In [15]:
np.random.seed(0)

pts = list()
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10, 2))
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)

pts = np.stack(pts, axis=0).astype(np.float32)
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

## 모델 생성

In [16]:
model = MyModel()

## 손실 함수 및 최적화 알고리즘 설정
### CrossEntropy, Adam Optimizer

In [17]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정
### Accuracy

In [18]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프

In [19]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 2.317021608352661, Accuracy: 12.5
Epoch 2, Loss: 1.975708246231079, Accuracy: 40.0
Epoch 3, Loss: 1.709601879119873, Accuracy: 53.500003814697266
Epoch 4, Loss: 1.508541464805603, Accuracy: 55.80000305175781
Epoch 5, Loss: 1.3689912557601929, Accuracy: 66.0
Epoch 6, Loss: 1.26003098487854, Accuracy: 71.5
Epoch 7, Loss: 1.1671955585479736, Accuracy: 74.69999694824219
Epoch 8, Loss: 1.0927270650863647, Accuracy: 78.30000305175781
Epoch 9, Loss: 1.036090612411499, Accuracy: 78.30000305175781
Epoch 10, Loss: 0.9698294997215271, Accuracy: 81.9000015258789
Epoch 11, Loss: 0.9156206846237183, Accuracy: 80.29999542236328
Epoch 12, Loss: 0.8736617565155029, Accuracy: 81.19999694824219
Epoch 13, Loss: 0.8390908241271973, Accuracy: 84.5
Epoch 14, Loss: 0.7995463013648987, Accuracy: 85.19999694824219
Epoch 15, Loss: 0.771921694278717, Accuracy: 85.29999542236328
Epoch 16, Loss: 0.7393359541893005, Accuracy: 85.39999389648438
Epoch 17, Loss: 0.7047868967056274, Accuracy: 85.0
Epoch 1

## 데이터셋 및 학습 파라미터 저장

In [20]:
np.savez_compressed('ch2_dataset.npz', inputs=pts, labels=labels)

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ch2_parameters.npz',
                    W_h=W_h,
                    b_h=b_h,
                    W_o=W_o,
                    b_o=b_o)